# 1. 

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

cpu is available


학습 전 준비하는 단계이다. <br>
필요한 모듈을 임포트 시켜준 후 multi-gpu 사용을 위해 세팅해준다. 더 빠른 딥러닝 학습을 위해서이다.<br>
딥러닝 학습 전 파라미터 튜닝을 해줘야한다. <br>
학습률 (learning_rate), 배치의 크기 (batch_size), epoch 수 (epochs), 클래스의 개수 (num_classes) 파라미터를 조정해준다.

# 2.

In [8]:
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)
 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
 
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

MNIST 데이터셋을 불러오는 단계이다. <br>
훈련 데이터 셋과 학습 데이터 셋을 각각 불러와서 train_set, test_set 변수에 할당해준다.<br>
생성시에는 0에서 255까지 있는 값을 0에서 1사이의 값으로 변환해준다. <br>
불러온 데이터를 DataLoader에 로드한다. 데이터 로더를 사용하여 배치 크기를 지정해준다. <br>
데이터 셋의 사이즈를 확인한다. 

# 3.

In [9]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() # super로 nn.Module 상속받는다.

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 파라미터는 <입력 채널 수 = 1, 출력 채널 수 = 10, kernel size = 5, stribe = 1, padding = 0, padding_mode = 'zeros'>이다. 
                                                    # conv1 함수를 선언하며, 채널수를 1에서 10으로 늘린다.  
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 파라미터는 <입력 채널 수 = 10, 출력 채널 수 = 20, kernel size = 5, stribe = 1, padding = 0, padding_mode = 'zeros'>이다. 
                                                      # conv2 함수를 선언하며, 채널수를 10에서 12로 늘린다. 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # dropout은 신경망에서 유닛을 제거(또는 드롭아웃)을 하여 다수의 아키텍처를 동시에 시뮬레이션하는 머신러닝 기술이다. 과접학될 가능성을 획기적으로 줄일 수 있다.
                                                           # 파라미터는 <요소가 0이 될 확률 p = 0.25, 작업 제자리에서 수행 = False >이다. drop2D 함수를 선언한다.
        self.mp = nn.MaxPool2d(2)  # 여러 입력 평면으로 구성된 입력 신호에 2D 최대 풀링을 적용한다.
                                   # 파라미터는 <kernel_size 최대를 차지할 창의 크기 = 2, 창의 보폭 = 2, 양쪽에 추가할 암시적 제로 패딩 = 0, 창에서 요소의 보폭을 제어하는 매개변수 =1, 최대 인덱스 반환=False, floor 사용해 출력 모양을 계산=False>이다.
        self.fc1 = nn.Linear(320,100) # 320개에서 100개의 출력으로 변경해준다.
        self.fc2 = nn.Linear(100,10)  # 100개에서 10개의 출력으로 변경해준다.

  def forward(self, x): # 신경망에 데이터를 지나가게 하는 forward 함수를 정의해준다.
                        # x는 데이터를 의미한다.
        x = F.relu(self.mp(self.conv1(x))) # x가 conv1을 지나가는 순서대로 
                                            # relu 활성함수(rectified-linear activation function)에 대입하고 mp 오버피팅을 방지해준다.
                                            # 결과값은 12X12X10이다.
        x = F.relu(self.mp(self.conv2(x))) # x가 conv2을 지나가는 순서대로 
                                            # relu 활성함수(rectified-linear activation function)에 대입하고 mp 오버피팅을 방지해준다.
                                            # 결과값은 4X4X20이다.
        x = self.drop2D(x) # x를 dropout시켜준다. 
        x = x.view(x.size(0), -1) # view를 이용하여 x를 flat하게 펼쳐준다.
        x = self.fc1(x) # fc1 레이어에 삽입해준다.
        x = self.fc2(x) # fc2 레이어에 삽입해준다.
        return F.log_softmax(x) # x를 fully-connected layer에 넣고 logsoftmax 적용한다.

모델 선언 전 클래스를 설계하는 단계이다. <br>
ConvNet 클래스로 model 안에 __init__, forward 함수를 만들고, nn.Module을 상속받아서 neural network를 생성해준다.

# 4.

In [10]:
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

모델을 정의하는 단계이다. <br>
ConvNet()으로 모델을 선언하고, to(device)사용해 gpu로 넘겨준다. <br>
그리고 비용 함수와 옵티마이저를 정의한다. <br>
PyTorch에서는 softmax와 cross-entropy를 합쳐놓은 것 을 제공한다.<br>
CrossEntropyLoss 함수의 default 파라미터 값은 <weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean'>이다.<br>
옵티마이저는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정이다.<br>
하이퍼 파라미터로는 학습하려는 모델의 매개변수와 학습률(learning rate)을 넣어주어 옵티마이저를 초기화시킨다.

# 5.

In [13]:
 for epoch in range(epochs): # for문을 사용하여 epoch의 수만큼 반복해준다.
    avg_cost = 0  # avg_cost에 누적해줘야 해서 0으로 시작한다.

    for data, target in train_loader: # train_data - train_target 순으로 train_loader에 저장되어 있으므로, 순서대로 지정해준다.
        data = data.to(device) # gpu를 활용하기 위해서 to(device)사용한다.
        target = target.to(device) # gpu를 활용하기 위해서 to(device)사용한다.
        optimizer.zero_grad() # zero_grad()를 사용하여 모든 모델의 gradient 값을 0으로 설정해준다.
                            # PyTorch에서는 미분한 값들이 누적되는 특징이 있기 때문에 초기화시켜줘야 한다.
                            # gradients를 0으로 초기화해주지 않으면, gradient가 의도한 방향과 달라져 학습이 원하는 방향으로 이루어지지 않을 수 있다.
        hypothesis = model(data) # 모델의 결과값을 hypothesis에 저장해준다.
        cost = criterion(hypothesis, target) # CrossEntropyLoss()에 의한 hypothesis와 target의 차이를 cost에 저장해준다.
        cost.backward() # backward()를 사용하여 gradient 계산해준다.
        optimizer.step() # 모델의 옵티마이저를 갱신해준다.
        avg_cost += cost / len(train_loader) # avg_cost에 loss를 누적해주며 train_loader 개수로 나누어 평균을 구해준다.
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost)) # 평균 loss 출력해준다.

C:\Users\shyun\AppData\Local\Temp/ipykernel_11720/3741363130.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1] cost = 0.0384443924
[Epoch:    2] cost = 0.0323067904
[Epoch:    3] cost = 0.0280739293
[Epoch:    4] cost = 0.0246941522
[Epoch:    5] cost = 0.0219674539


모델을 학습하는 단계이다. <br>
epoch 수만큼 모델이 전체 Training 데이터 세트를 훈련시켜준다.<br>
cost 낮을수록 좋은 모델이다. <br>
epoch 수가 5개일 때, cost값이 0.0661975741로 가장 낮다.

# 6.

In [12]:
model.eval() # .eval() 함수는 평가과정에서 사용하지 않아야 하는 layer들을 알아서 종료시키도록 하는 함수이다.
             # 모델 학습을 종료한다.
with torch.no_grad():  # no_grad()로 gradient 트래킹을 하지 않도록 해준다.
    correct = 0 # 정확도
    total = 0 # 진행률

    for data, target in test_loader: # test_data - test_target 순으로 test_loader에 저장되어 있으므로, 순서대로 지정해준다.
        data = data.to(device) # gpu를 활용하기 위해서 to(device)사용한다.
        target = target.to(device) # gpu를 활용하기 위해서 to(device)사용한다.
        out = model(data) # 학습된 모델의 결과값을 out에 저장해준다.
        preds = torch.max(out.data, 1)[1] # preds에 가장 높은 값의 인덱스로 지정해준다.
        total += len(target) # 진행률 누적시켜준다. 
        correct += (preds==target).sum().item() # 정확도를 계산해준다. 
        
    print('Test Accuracy: ', 100.*correct/total, '%') # 모델의 학습 정도를 출력해준다.

C:\Users\shyun\AppData\Local\Temp/ipykernel_11720/3741363130.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy:  98.54 %


모델을 평가하는 단계이다. <br>
모델의 학습 정도를 확인할 수 있다.